实例57的爬虫程序已经一次性获取了想要的文章，后续还会不定期的下载新文章，那怎么办呢？不可能每次都运行那个程序，然后重新从头到尾下载一遍吧。那样也忒傻了吧，坚决不干傻事。咱稍稍修改一个“更新爬虫”程序，只需双击，有新文章则自动下载，没新文章则原地不动。
<br/>
<br/>理理思路先：
1. 将上次获取的4946篇文章的链接作为一个数据库存在一个文本文件中
2. 后续只获取最新的前4页的文章链接，若链接已存在，则跳过
3. 若链接不存在，则加入数据库，并下载该链接里的文章。
<br/>
<br/>于是乎，开始按这个思路去整理程序。

In [ ]:
#将所有历史文章的链接存入文本文件中
import requests
import json

header = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.75 Safari/537.36'}

form_data = {'_q': 'Article.list',
'siteId': '7e0b3b27-2622-4aa7-b6f8-abfe5c5df922',
'catalogId': '34f92da3-d6d0-4e96-899f-d7f581c18162',
'pub': 'true',
'limit': 10000,
'start': 1}

#这是异步加载，请求方法是POST
url = "http://www.bicpa.org.cn/dtzj/zxgg/getArticles.action"
res = requests.post(url, data = form_data, headers = header)
article_data0 =res.text.split("{success:true,datas:")[1] #去掉字符串前面的无用信息“{success:true,datas:”
article_data = article_data0.split(",total:")[0] #去掉字符串后面的无用信息“,total:4946}”
obj = json.loads(article_data)

#将所有文章链接写入文本文件
path = r"http://www.bicpa.org.cn"
links_file = open('links.txt', 'a') #避免覆盖已有数据，用添加模式`a`写入
for info in obj:
    link = path + info['url'] + info['primaryKey']+".html\n" #只提取链接信息，并换行            
    links_file.write(link) #写入链接信息
links_file.close() #写完后关闭

以上，只是在实例57的“获取所有文章链接”的基础上增加了“将链接写入文本文件”的程序。我们写入的文本文件名是“links.txt”，写入方式为“添加模式”，即程序中的“a”，这样才不会覆盖已有数据。写好的文本文件的内容如下。
![](images\links.png)

现在，忘记历史，开始新的征程。假设过了一周，查看一下目标网站有无新的文章。判断标准就是看这篇文章的链接是否已存在“links.txt”文件中。如果有新文章，则将新的链接存入数据库“links.txt”中，并判断这篇文章的标题是否含有关键词“委员会专家提示”，若含有则下载到本地；若不含有，则不下载。如果目标网站没有新的文章，则啥也不做。写入文章到word文件的函数`Get_article_to_word`，直接照抄，随时待命被调用。

In [11]:
#定义函数，获取想要的文章并批量写入word文件
import requests
from bs4 import BeautifulSoup
import docx
from docx.shared import Pt #用于设定字体大小（磅值）
from docx.oxml.ns import qn #用于应用中文字体
import random
import time

def Get_article_to_word(url,date):
    user_agent_list = ["Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36",
                    "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36",
                    "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36",
                    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.62 Safari/537.36",
                    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/45.0.2454.101 Safari/537.36",
                    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0)",
                    "Mozilla/5.0 (Macintosh; U; PPC Mac OS X 10.5; en-US; rv:1.9.2.15) Gecko/20110303 Firefox/3.6.15"
                    ]
    header = {'User-Agent': user_agent_list}
    header['User-Agent'] = random.choice(user_agent_list) #每篇文章随机选择浏览器，避免单个浏览器请求太快被服务器切断连接
    wb_data = requests.get(url,headers = header)
    soup = BeautifulSoup(wb_data.content)
    title = soup.select('.headword')[0].text.strip()#获得标题
    content1 = soup.select(".MsoNormal") #针对正文布局为 class = "MsoNormal"
    content2 = soup.select("#art_content") #针对正文布局为 id = "art_content"
    
    doc = docx.Document() #新建空白word文档
    #设定全局字体
    doc.styles['Normal'].font.name=u'宋体'
    doc.styles['Normal']._element.rPr.rFonts.set(qn('w:eastAsia'), u'宋体')
   
    #写入标题行，并设置字体格式
    p = doc.add_paragraph()
    r = p.add_run(title)
    r.bold = True
    r.font.size = Pt(18)
    
    doc.add_paragraph(date) #写入日期
    doc.add_paragraph(url) #写入文章链接
    
    #写入正文
    for i in content2:
        doc.add_paragraph(i.text)
    for i in content1:
        doc.add_paragraph(i.text)
    doc.save(f"文章\\{title}.docx")

然后写判断网站是否有新文章更新的程序。获取文章链接的部分基本照抄，只是将`form_data`里的`limit`改成150。因为这个网站更新速度较慢，每次查看是否有更新的时候，只获取前10页共150篇文章的信息就足够了。
<br/>
<br/>最近的60篇文章的链接获取好后，就需要载入数据库中保存的链接，作为判断是否有新文章的基准。使用读取模式(`"r"`)打开，并用`read()`方法读取成一个大字符串，存入`link_database`变量。
<br/>
<br/>然后以添加模式“a”打开数据库文件`links`，以便有新的链接时进行写入操作。然后新建两个初始值为0的计数器，`counter_link`和`counter_download`，分别记录新文章链接个数和新下载的文章个数。
<br/>
<br/>遍历新获取的60篇文章的信息，拼接好链接信息`link`。然后判断其是否已存在字符串`link_database`里面，如果已存在，则飘过(`pass`)，直接回到`for`循环，检查下一篇文章的信息。如果不存在字符串`link_database`里面，则表示是新发布的文章，需要将其链接加入数据库。然后再进一步判断这篇文章的标题是否含有关键词“委员会专家提示”，如果含有，则下载到本地word文档；若不含有，则不下载。
<br/>
<br/>每增加一条新文章的链接到数据库文件，`counter_link`增加1。每下载一篇需要的文章，`counter_download`增加1。最后根据这两个计数器的值，来显示新文章的获取情况。

In [14]:
#判断网站是否有新文章
import requests
import json

header = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.75 Safari/537.36'}
form_data = {'_q': 'Article.list',
'siteId': '7e0b3b27-2622-4aa7-b6f8-abfe5c5df922',
'catalogId': '34f92da3-d6d0-4e96-899f-d7f581c18162',
'pub': 'true',
'limit': 150, #网站更新速度较慢，每次查看是否有更新的时候，只获取前10页共150篇文章的信息
'start': 1}

#这是异步加载，请求方法是POST
url = "http://www.bicpa.org.cn/dtzj/zxgg/getArticles.action"
res = requests.post(url, data = form_data, headers = header)
article_data0 =res.text.split("{success:true,datas:")[1] #去掉字符串前面的无用信息“{success:true,datas:”
article_data = article_data0.split(",total:")[0] #去掉字符串后面的无用信息“,total:xxxx}”
obj = json.loads(article_data)

#载入数据库中的所有链接，作为判断基准
f = open('links.txt', 'r')
link_database = f.read() #读取成一个大字符串

#将所有新文章链接写入文本文件
path = r"http://www.bicpa.org.cn"
links_file = open('links.txt', 'a') #避免覆盖已有数据，用添加模式`a`写入
counter_link = 0 #新文章链接计数器
counter_download = 0 #新下载文章计数器
for info in obj:
    link = path + info['url'] + info['primaryKey']+".html" #拼接链接信息
    if link in link_database: #判断新提取的链接是否已存在于数据库
        pass
    else:
        links_file.write(link+"\n") #写入链接信息
        counter_link +=1 
        
        #下载标题含有"委员会专家提示"的文章
        if "委员会专家提示" in info['title']:
            Get_article_to_word(link,info["publishDate"]) #调用写好的函数，下载文章到word文件
            counter_download += 1       #每下载一篇文章，计数器增加1
        
links_file.close() #写完后关闭文件

#显示每次的结果
if counter_link == 0:
    print("没有文章更新！")
else:
    print(f"共获取到 {counter_link} 篇新文章的链接，并加入数据库。")

if counter_download == 0:
    print("没有'委员会专家提示'文章更新！")
else:
    print(f"共下载 {counter_download} 篇'委员会专家提示'新文章，请到文件夹查看。")

共获取到 125 篇新文章的链接，并加入数据库。
共下载 3 篇'委员会专家提示'新文章，请到文件夹查看。


到此，任务圆满完成！但是我们想看看有新文章出现，程序是否能正常运行。好办，将`links.txt`中最前面的链接数据删除几十条，再运行程序即可。程序运行结果显示：
```
共获取到 125 篇新文章的链接，并加入数据库。
共下载 3 篇'委员会专家提示'新文章，请到文件夹查看。
```
Good!以后想查看有无文章更新，只需要双击一下这个程序就可以了，真个方便得不要不要的。